<a href="https://colab.research.google.com/github/spsaswat/plantdis/blob/main/test_pd_village_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile
import os


In [2]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Check the contents of the drive to ensure the dataset is accessible
!ls /content/drive/My\ Drive/Colab\ Notebooks/Plant_leave_diseases_dataset_sampled


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Apple___Apple_scab			       Pepper,_bell___healthy
 Apple___Black_rot			       Potato___Early_blight
 Apple___Cedar_apple_rust		       Potato___healthy
 Apple___healthy			       Potato___Late_blight
 Blueberry___healthy			       Raspberry___healthy
 Cherry___healthy			       Soybean___healthy
 Cherry___Powdery_mildew		       Squash___Powdery_mildew
'Corn___Cercospora_leaf_spot Gray_leaf_spot'   Strawberry___healthy
 Corn___Common_rust			       Strawberry___Leaf_scorch
 Corn___healthy				       Tomato___Bacterial_spot
 Corn___Northern_Leaf_Blight		       Tomato___Early_blight
 Grape___Black_rot			       Tomato___healthy
'Grape___Esca_(Black_Measles)'		       Tomato___Late_blight
 Grape___healthy			       Tomato___Leaf_Mold
'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)'   Tomato___Septoria_leaf_spot
'Orange___Haunglongbing_(Citrus_greening)'    'Tomato___Spider_

In [3]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

# Load the TensorFlow Lite model
tflite_model_path = 'pd_village.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)

# Allocate tensors
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

# Function to preprocess the input image
def preprocess_image(image_path):
    img = Image.open(image_path).resize((224, 224))
    img = np.array(img).astype(np.float32) / 255.0  # Normalize to [0, 1]
    return np.expand_dims(img, axis=0)  # Add batch dimension

# Path to the sampled dataset directory in Google Drive
dataset_dir = '/content/drive/My Drive/Colab Notebooks/Plant_leave_diseases_dataset_sampled'

# List of class names (folder names)
class_names = [
    'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy',
    'Blueberry___healthy', 'Cherry___healthy', 'Cherry___Powdery_mildew',
    'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___healthy',
    'Corn___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)',
    'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy',
    'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight',
    'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy',
    'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch',
    'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy',
    'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot',
    'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
]

# Counters for correct and incorrect predictions
correct_count = 0
incorrect_count = 0
total_images = 0

# Iterate through each class folder
for class_name in class_names:
    class_folder_path = os.path.join(dataset_dir, class_name)

    if os.path.isdir(class_folder_path):
        for image_name in os.listdir(class_folder_path):
            image_path = os.path.join(class_folder_path, image_name)

            # Preprocess the image
            preprocessed_image = preprocess_image(image_path)

            # Set the input tensor
            interpreter.set_tensor(input_details[0]['index'], preprocessed_image)

            # Invoke the interpreter
            interpreter.invoke()

            # Get the output tensor
            output_data = interpreter.get_tensor(output_details[0]['index'])
            logits = output_data[0]

            # Get the predicted class index
            predicted_class_index = np.argmax(logits)
            predicted_class_name = class_names[predicted_class_index]

            # Compare the predicted class with the actual class
            if predicted_class_name == class_name:
                correct_count += 1
            else:
                incorrect_count += 1

            # Print the result
            print(f"Image: {image_path}")
            print(f"Actual label: {class_name}")
            print(f"Predicted label: {predicted_class_name}")
            print(f"Correct: {predicted_class_name == class_name}\n")

            total_images += 1

# Print the final results
print(f"Total images: {total_images}")
print(f"Correct predictions: {correct_count}")
print(f"Incorrect predictions: {incorrect_count}")
print(f"Accuracy: {correct_count / (correct_count + incorrect_count):.2f}")


Streaming output truncated to the last 5000 lines.

Image: /content/drive/My Drive/Colab Notebooks/Plant_leave_diseases_dataset_sampled/Raspberry___healthy/image (499).JPG
Actual label: Raspberry___healthy
Predicted label: Raspberry___healthy
Correct: True

Image: /content/drive/My Drive/Colab Notebooks/Plant_leave_diseases_dataset_sampled/Raspberry___healthy/image (384).JPG
Actual label: Raspberry___healthy
Predicted label: Raspberry___healthy
Correct: True

Image: /content/drive/My Drive/Colab Notebooks/Plant_leave_diseases_dataset_sampled/Raspberry___healthy/image (640).JPG
Actual label: Raspberry___healthy
Predicted label: Raspberry___healthy
Correct: True

Image: /content/drive/My Drive/Colab Notebooks/Plant_leave_diseases_dataset_sampled/Raspberry___healthy/image (874).JPG
Actual label: Raspberry___healthy
Predicted label: Soybean___healthy
Correct: False

Image: /content/drive/My Drive/Colab Notebooks/Plant_leave_diseases_dataset_sampled/Raspberry___healthy/image (202).JPG
Actua